In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
64,ABW,Aruba,2020-05-26,101,0,3,0,945.994,0.000,28.099,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
202,AFG,Afghanistan,2020-05-26,11173,591,219,1,287.015,15.182,5.626,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
268,AGO,Angola,2020-05-26,69,0,4,0,2.099,0.000,0.122,...,2.405,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-25,France,145145.0,0.0,28419.0,0.0,0.0,0.0
0,2020-05-25,China,84164.0,0.0,4655.0,0.0,0.0,0.0
0,2020-05-25,Italy,230617.0,0.0,32918.0,0.0,0.0,0.0
0,2020-05-25,Spain,236249.0,0.0,28811.0,0.0,0.0,0.0
0,2020-05-25,United States,1666456.0,0.0,98686.0,0.0,0.0,0.0
0,2020-05-25,World,5484451.0,0.0,349227.0,0.0,0.0,0.0
0,2020-05-25,United Kingdom,262800.0,0.0,37105.0,0.0,0.0,0.0
0,2020-05-25,Germany,179060.0,0.0,8281.0,0.0,0.0,0.0
0,2020-05-25,Iran,137684.0,0.0,7482.0,0.0,0.0,0.0
0,2020-05-25,Turkey,157941.0,0.0,4369.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-26,France,145145.0,145279.0,28419.0,28432.0,-134.0,-13.0
0,2020-05-26,China,84164.0,84102.0,4655.0,4638.0,62.0,17.0
0,2020-05-26,Italy,230617.0,230158.0,32918.0,32877.0,459.0,41.0
0,2020-05-25,Spain,236249.0,235400.0,28811.0,26834.0,849.0,1977.0
0,2020-05-26,United States,1666456.0,1662302.0,98686.0,98220.0,4154.0,466.0
0,2020-05-26,World,5484451.0,5459526.0,349227.0,345994.0,24925.0,3233.0
0,2020-05-26,United Kingdom,262800.0,261184.0,37105.0,36914.0,1616.0,191.0
0,2020-05-26,Germany,179060.0,179002.0,8281.0,8302.0,58.0,-21.0
0,2020-05-26,Iran,137684.0,137724.0,7482.0,7451.0,-40.0,31.0
0,2020-05-26,Turkey,157941.0,157814.0,4369.0,4369.0,127.0,0.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")